In [1]:
import pandas as pd
import numpy as np
traindf = pd.read_csv('train.csv')
input_datadf = pd.read_csv('test.csv')

In [2]:
valid_size = 20000
train_size = len(traindf) - valid_size
train_seq = [map(long,traindf['Sequence'].loc[i].split(',')) for i in xrange(train_size)]
valid_seq = [map(long,traindf['Sequence'].loc[i].split(',')) for i in xrange(train_size,train_size+valid_size)]        
print(len(train_seq),len(valid_seq))
print(train_seq[0])
print(valid_seq[0])

(93845, 20000)
[1L, 3L, 13L, 87L, 1053L, 28576L, 2141733L, 508147108L, 402135275365L, 1073376057490373L, 9700385489355970183L, 298434346895322960005291L, 31479360095907908092817694945L, 11474377948948020660089085281068730L]
[2L, 1L, 5L, 2L, 1L, 5L, 2L, 1L, 4L, 1L, 1L, 3L, 2L, 1L, 1L, 2L, 1L, 2L, 2L, 1L, 5L, 1L, 1L, 5L, 1L, 1L, 4L, 2L, 1L, 1L, 2L, 1L, 3L, 2L, 1L, 1L, 2L, 1L, 2L, 1L, 1L, 4L, 1L, 1L, 1L, 2L, 1L, 5L, 2L, 1L, 2L, 2L, 1L, 2L, 1L, 1L, 1L, 1L, 1L, 1L, 2L, 1L, 5L, 1L, 1L, 4L, 2L, 1L, 1L, 1L, 1L, 3L, 2L, 1L, 1L, 2L, 1L, 1L, 2L, 1L, 2L, 1L, 1L, 2L, 1L, 1L, 3L, 2L, 1L, 1L, 1L, 1L, 2L, 2L, 1L, 3L, 2L, 1L, 1L, 1L]


In [3]:
y_valid = []
for i in xrange(valid_size):
    if len(valid_seq[i]) == 1:
        y_valid.append(valid_seq[i][-1])
    else:
        y_valid.append(valid_seq[i].pop())
print(y_valid[:10])

[1L, 32L, 71L, 4889L, 2269L, 342174928102L, 1L, 1939137135947326L, 71366238377013998880L, 323L]


In [4]:
train_shift, valid_shift= [], []
train_shift = [train_seq[i][j:j+window] for i in xrange(train_size) for window in xrange(3,len(train_seq[i]))
              for j in xrange(len(train_seq[i])-window+1)]
print('ck1')
valid_shift = [valid_seq[i][j:j+window] for i in xrange(valid_size) for window in xrange(3,len(valid_seq[i]))
              for j in xrange(len(valid_seq[i])-window+1)]
print(len(train_shift),len(valid_shift))

ck1
(112490283, 23482823)


In [ ]:
import marisa_trie
import fractions
def findGCD(seq):
    gcd = seq[0]
    for i in xrange(1,len(seq)):
        gcd = fractions.gcd(gcd,seq[i])
    return abs(gcd)

def findSignature(seq):
    nonzero_seq = [d for d in seq if d!=0]
    if len(nonzero_seq)==0:
        return seq, 0
    sign = 1 if nonzero_seq[0]>0 else -1
    gcd = findGCD(seq)
    return [sign*x//gcd for x in seq], sign*gcd

In [ ]:
train_str_seq, valid_str_seq= [], []
train_shift_str_seq, valid_shift_str_seq =  [], []
str_seq = []

for i in xrange(len(train_seq)):
    signature, _ = findSignature(train_seq[i])
    str_seq.append(','.join(map(str, signature)))
for i in xrange(len(valid_seq)):
    signature, _ = findSignature(valid_seq[i])
    str_seq.append(','.join(map(str, signature)))
# for i in xrange(len(test_seq)):
#     signature, _ = findSignature(test_seq[i] + [y_test[i]])
#     test_str_seq.append(','.join(map(str, signature)))

for i in xrange(len(train_shift)):
    signature, _ = findSignature(train_shift[i])
    str_seq.append(','.join(map(str, signature)))
del train_shift
for i in xrange(len(valid_shift)):
    signature, _ = findSignature(valid_shift[i])
    str_seq.append(','.join(map(str, signature)))
del valid_shift
# for i in xrange(len(test_shift)):
#     signature, _ = findSignature(test_shift[i])
#     test_shift_str_seq.append(','.join(map(str, signature)))

    
def Construct_trie(data_id=1):
#     1:train 2:+vaild 3:+valid+test 4:3 + input

    if data_id == 1: 
        str_seq = train_str_seq
    elif data_id == 2:
        str_seq = train_str_seq + valid_str_seq + train_shift_str_seq + valid_shift_str_seq 
    trie = marisa_trie.Trie(str_seq)
    count_signature = np.zeros(len(trie.items()), dtype = int)
    for i in xrange(len(str_seq)):
        count_signature[trie.key_id(unicode(str_seq[i]))] += 1
    return trie,count_signature

In [ ]:
trie,count_signature = Construct_trie(data_id=2)
print(len(trie.items()),count_signature[:10])

In [16]:
print(trie.restore_key(292476))

1,1,1,1


y_pred into dataframe and output csv#### 

In [ ]:
def checkRecurrence(seq, order= 2, minlength = 5):
    """
    :type seq: List[int]
    :type order: int
    :type minlength: int 
    :rtype: List[int]
    
    Check whether the input sequence is a recurrence sequence with given order.
    If it is, return the coefficients for the recurrenec relation.
    If not, return None.
    """     
    if len(seq)< max((2*order+1), minlength):
        return None
    
    ################ Set up the system of equations 
    A,b = [], []
    for i in xrange(order):# style: use list to append and then transform to numpy
        A.append(seq[i:i+order])
        b.append(seq[i+order]) 
    A,b =np.array(A), np.array(b)
    try: 
        if np.linalg.det(A)==0: #no inverse
            return None
    except TypeError:
        return None
   
    #############  Solve for the coefficients (c0, c1, c2, ...)
    coeffs = np.linalg.inv(A).dot(b)  # x = inv(A) dot b
    
    ############  Check if the next terms satisfy recurrence relation
    for i in xrange(2*order, len(seq)):
        predict = np.sum(coeffs*np.array(seq[i-order:i]))
        if abs(predict-seq[i])>10**(-2): # if error < 0.01 return None.
            return None
    
    return list(coeffs)


def predictNextTerm(seq, coeffs):
    """
    :type seq: List[int]
    :type coeffs: List[int]
    :rtype: int
    
    Given a sequence and coefficienes, compute the next term for the sequence.
    """
    
    order = len(coeffs)
    predict = np.sum(coeffs*np.array(seq[-order:]))
    return long(round(predict))


In [ ]:
import random
def checkGP(seq):
    if(len(seq) <= 2):
        return None
    div_seq = []
    for i in xrange(len(seq)-1):
        if seq[i] == 0:
            return None
        else:
            if seq[i+1] % seq[i] != 0:
                return None
            div_seq.append(seq[i+1] / float(seq[i]))
    if np.all(np.diff(div_seq,1) == 0):
        return long(seq[-1] * (seq[-1] / float(seq[-2]))) #return value is int 
    else:
        return None

def findDerivative(seq):
    return [0] if len(seq)<=1 else [seq[i]-seq[i-1] for i in xrange(1,len(seq))]

def Newton_diff(seq):
    tail = []
    for i in xrange(len(seq)-1):
        if len(set(seq)) == 1:
            return sum(tail) + seq[-1]
        tail.append(seq[-1])
        seq = findDerivative(seq)
    return None

def math_diff(seq):
    pred = Newton_diff(seq)
    if pred != None:
        return pred
    pred = checkGP(seq)
    if pred != None:
        return pred
    for order in xrange(2,(len(seq)+1)/2):
        coeffs = checkRecurrence(seq, order)
        if coeffs != None:
            pred = predictNextTerm(seq, coeffs)
            break
    return pred

def trie_diff(seq):
    random.seed(4)
    pred = None
    pred_candidate = []
#     length left at least 1
    for i in xrange(len(seq)):
        shift_seq = seq[i:]
        signature, gcd = findSignature(shift_seq) 
        key = unicode(','.join(map(str,signature)) + ',')
        prefixes_in_trie = trie.keys(key)
        if prefixes_in_trie != []:
            start_index = len(key)
            count_prefix = [count_signature[trie.key_id(x)] for x in prefixes_in_trie]
            _ = max(count_prefix)
            index_cand = [x for x in xrange(len(count_prefix)) if count_prefix[x] == _]
            index = random.choice(index_cand)
            end_index = prefixes_in_trie[index].find(',',start_index)
            if end_index == -1:
                end_index = None
            pred_str = prefixes_in_trie[index][start_index:end_index]
            if pred_str != '':
                pred = long(pred_str) * gcd
                return pred
#                     pred_candidate.append(pred,len(shift_seq))
#     pred_candidate = sorted(pred_candidate, key=lambda x: (-x[1],x[2]))
#     print(pred_candidate)
#     if pred_candidate != []:
#         return pred_candidate[0][0]
    return None

def pred_math_trie(seqs):
#     train_id = 0, valid_id = 1, test_id = 2, input_id = 3
    count = 0
    seqs_pred, record = np.empty(len(seqs),dtype = object), np.empty(len(seqs),dtype = (str,10))
    for i in xrange(len(seqs)):
        flag = 'math'
        pred = math_diff(seqs[i])
        if pred == None:
            pred = trie_diff(seqs[i])
            flag = 'trie'
        if pred != None:
            seqs_pred[i] = pred
            count += 1
            record[i] = flag
            print(i,pred,count,flag)
    return seqs_pred, record
_ = [2,2,4,6,10]
print(math_diff(_))

##### can't find [1,27,125,343] n**3 pattern (don't have enough terms)
##### divide by 0: [0, 0, 0, 0, 0]: difference will find out.
##### [7,0,0,0,0]: won't work.

In [ ]:
valid_subsize = 10000
valid_pred, record = pred_math_trie(valid_seq[:valid_subsize])
mask1, mask2 = record == 'math', record == 'trie'
math_accuracy = sum(valid_pred[mask1] == np.array(y_valid[:valid_subsize])[mask1]) * 100 / float(sum(mask1))\
                if sum(mask1) != 0 else -1
trie_accuracy = sum(valid_pred[mask2] == np.array(y_valid[:valid_subsize])[mask2]) * 100 / float(sum(mask2))\
                if sum(mask2) != 0 else -1
valid_accuracy = sum(valid_pred == y_valid[:valid_subsize]) * 100 / float(valid_subsize)

mask3 = np.logical_or(record == 'trie', record == 'math')
pred_accuracy = sum(valid_pred[mask3] == np.array(y_valid[:valid_subsize])[mask3]) * 100 / float(sum(mask3))\
                if sum(mask3) != 0 else -1

print('validation accuracy= %.2f %%') % valid_accuracy
print('math accuracy= %.2f %%') % math_accuracy
print('trie accuracy= %.2f %%') % trie_accuracy
print('pred accuracy= %.2f %%') % pred_accuracy

use trie[0] now. can improve. eg.max len
use int now. might have bug.
find trie might wrong .can improve
use checkdiff better than trie
don't use test for trie
don;t use signature now
[1,7;49] vs [1,7,13]
check gp for each diff might wrong
improve: boundary.